In [82]:
import open3d as o3d
import copy
import numpy as np
import single_image as si

ModuleNotFoundError: No module named 'single_image'

In [76]:
def draw_registration_result(source, target, transformation):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    #source_temp.paint_uniform_color([1, 0.706, 0])
    #target_temp.paint_uniform_color([0, 0.651, 0.929])
    source_temp.transform(transformation)
    pcd_plus = source_temp + target_temp
    # o3d.visualization.draw_geometries([source_temp, target_temp],
    #                                   zoom=0.4559,
    #                                   front=[0.6452, -0.3036, -0.7011],
    #                                   lookat=[1.9892, 2.0208, 1.8945],
    #                                   up=[-0.2779, -0.9482, 0.1556])
    #o3d.visualization.draw_geometries([pcd_plus])
    return pcd_plus

In [77]:
def preprocess_point_cloud(pcd,voxel_size):
    print(":: Downsample with a voxel size %.3f." % voxel_size)
    pcd_down = pcd.voxel_down_sample(voxel_size)

    radius_normal = voxel_size * 2
    print(":: Estimate normal with search radius %.3f." % radius_normal)
    pcd_down.estimate_normals(
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=30))

    radius_feature = voxel_size * 5
    print(":: Compute FPFH feature with search radius %.3f." % radius_feature)
    pcd_fpfh = o3d.pipelines.registration.compute_fpfh_feature(
        pcd_down,
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=100))
    return pcd_down, pcd_fpfh

In [78]:
def global_registration(source,target,voxel_size):
    target_down, target_fpfh = preprocess_point_cloud(target, voxel_size)
    source_down, source_fpfh = preprocess_point_cloud(source, voxel_size)
    distance_threshold = voxel_size * 1.5
    result = o3d.pipelines.registration.registration_ransac_based_on_feature_matching(
        source_down, target_down, source_fpfh, target_fpfh, True,
        distance_threshold,
        o3d.pipelines.registration.TransformationEstimationPointToPoint(True),
        3, [
            o3d.pipelines.registration.CorrespondenceCheckerBasedOnEdgeLength(
                0.9),
            o3d.pipelines.registration.CorrespondenceCheckerBasedOnDistance(
                distance_threshold)
        ], o3d.pipelines.registration.RANSACConvergenceCriteria(100000, 0.999))
    ransac_result = draw_registration_result(source_down, target_down, result.transformation)
    return ransac_result

In [79]:
target = o3d.io.read_point_cloud("D:/sem2/intern/workspace/apple/x_pcd/1/pcd0.pcd")
for i in [11,23,68,80,90,111,125]:
    voxel_size=round(max(target.get_max_bound()-target.get_min_bound())*0.005,4)
    source = o3d.io.read_point_cloud("D:/sem2/intern/workspace/apple/x_pcd/1/pcd%d.pcd" %i)
    target = global_registration(source,target,voxel_size)
o3d.visualization.draw_geometries([target])
    

:: Downsample with a voxel size 0.002.
:: Estimate normal with search radius 0.005.
:: Compute FPFH feature with search radius 0.011.
:: Downsample with a voxel size 0.002.
:: Estimate normal with search radius 0.005.
:: Compute FPFH feature with search radius 0.011.
:: Downsample with a voxel size 0.002.
:: Estimate normal with search radius 0.005.
:: Compute FPFH feature with search radius 0.011.
:: Downsample with a voxel size 0.002.
:: Estimate normal with search radius 0.005.
:: Compute FPFH feature with search radius 0.011.
:: Downsample with a voxel size 0.002.
:: Estimate normal with search radius 0.005.
:: Compute FPFH feature with search radius 0.011.
:: Downsample with a voxel size 0.002.
:: Estimate normal with search radius 0.005.
:: Compute FPFH feature with search radius 0.011.
:: Downsample with a voxel size 0.002.
:: Estimate normal with search radius 0.005.
:: Compute FPFH feature with search radius 0.011.
:: Downsample with a voxel size 0.002.
:: Estimate normal with

In [80]:
pcd_combined = target
alpha_mesh = si.alpha_mesh()
o3d.visualization.draw_geometries([alpha_mesh])

NameError: name 'si' is not defined